# Iterative Quantum Amplitude Estimation Demo

**Team QHackers** | GenQ Hackathon 2025

Demonstrates:
1. Distribution encoding in quantum states
2. Threshold oracle for CDF estimation
3. Binary search for α-quantile (PFE)
4. Discretization tradeoffs

In [ ]:
import sys
sys.path.append('../backend')

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from scipy.stats import norm

from app.services.quantum_qae import (
    discretize_normal_distribution,
    create_state_preparation_circuit,
    create_threshold_oracle,
    compute_pfe_quantum
)

plt.rcParams['figure.figsize'] = (12, 6)
SEED = 42

## 1. Discretize Distribution

Encode continuous distribution into discrete bins for quantum encoding.

In [ ]:
# Example: Discretize normal distribution
num_qubits = 5
num_bins = 2 ** num_qubits

mu = 5.0
sigma = 2.0

bin_centers, probabilities = discretize_normal_distribution(
    mu=mu, sigma=sigma, num_bins=num_bins, num_std=3.0
)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.bar(bin_centers, probabilities, width=(bin_centers[1]-bin_centers[0])*0.9, alpha=0.7)
x = np.linspace(mu-3*sigma, mu+3*sigma, 100)
ax1.plot(x, norm.pdf(x, mu, sigma), 'r--', linewidth=2, label='Continuous')
ax1.set_title(f'Discretized Distribution ({num_bins} bins)', fontweight='bold')
ax1.set_xlabel('Value')
ax1.set_ylabel('Probability Density')
ax1.legend()
ax1.grid(alpha=0.3)

# CDF
cdf = np.cumsum(probabilities)
ax2.step(bin_centers, cdf, where='mid', linewidth=2)
ax2.axhline(0.95, color='red', linestyle='--', label='α=0.95')
ax2.set_title('Cumulative Distribution', fontweight='bold')
ax2.set_xlabel('Value')
ax2.set_ylabel('P(X ≤ x)')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Number of qubits: {num_qubits}")
print(f"Number of bins: {num_bins}")
print(f"Bin width: {bin_centers[1] - bin_centers[0]:.3f}")

## 2. Quantum State Preparation

Create |ψ⟩ = Σ √p_i |i⟩

In [ ]:
# Create state preparation circuit
state_prep = create_state_preparation_circuit(probabilities, num_qubits)
print(f"State preparation circuit depth: {state_prep.depth()}")
print(f"Number of gates: {len(state_prep)}")
state_prep.draw('mpl')

## 3. Threshold Oracle

Marks states where value ≤ threshold

In [ ]:
# Create threshold oracle for median
threshold = mu
oracle = create_threshold_oracle(bin_centers, threshold, num_qubits)
print(f"Oracle circuit depth: {oracle.depth()}")
print(f"Number of gates: {len(oracle)}")

## 4. Full Quantum PFE Calculation

In [ ]:
# Portfolio parameters
result = compute_pfe_quantum(
    w1=0.5, w2=0.5,
    strike=100.0,
    s0=100.0,
    mu=0.05,
    sigma=0.2,
    tau=1.0,
    num_qubits=5,
    ae_iterations=6,
    alpha=0.95,
    seed=SEED
)

print("\nQuantum PFE Results:")
print(f"  PFE: {result['pfe']:.2f}")
print(f"  Expected Exposure: {result['expected_exposure']:.2f}")
print(f"  Qubits: {result['num_qubits']}")
print(f"  Discretization bins: {result['discretization_bins']}")
print(f"  Runtime: {result['runtime_ms']:.1f} ms")

## 5. Discretization Tradeoff Analysis

In [ ]:
# Test different qubit counts
qubit_range = [3, 4, 5, 6, 7]
pfe_results = []
runtimes = []

for q in qubit_range:
    res = compute_pfe_quantum(
        w1=0.5, w2=0.5, strike=100.0, s0=100.0,
        mu=0.05, sigma=0.2, tau=1.0,
        num_qubits=q, ae_iterations=6,
        alpha=0.95, seed=SEED
    )
    pfe_results.append(res['pfe'])
    runtimes.append(res['runtime_ms'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(qubit_range, pfe_results, 'o-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Qubits', fontsize=12)
ax1.set_ylabel('PFE Estimate', fontsize=12)
ax1.set_title('PFE vs Discretization', fontweight='bold')
ax1.grid(alpha=0.3)

ax2.plot(qubit_range, runtimes, 'o-', linewidth=2, markersize=8, color='green')
ax2.set_xlabel('Number of Qubits', fontsize=12)
ax2.set_ylabel('Runtime (ms)', fontsize=12)
ax2.set_title('Computational Cost', fontweight='bold')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

Key insights:
1. **Discretization** is necessary to encode continuous distributions
2. **Binary search** with threshold oracle finds quantiles efficiently  
3. **Tradeoff**: More qubits = better accuracy but higher circuit complexity
4. **Advantage**: Quantum converges as O(1/N) vs classical O(1/√N)